In [2]:
import pandas as pd

trainning_df = pd.read_pickle('../data/training_classif_data.pkl')
df_with_nan = pd.read_pickle('../data/df_with_nan.pkl')


In [ ]:
# Standardiser les données ????

# from sklearn.preprocessing import StandardScaler
# scaler = StandardScaler()
# Standardiser toutes les colonnes du DataFrame

# il faut encoder !! puis standardiser !!
# df = pd.DataFrame(scaler.fit_transform(df), columns=df.columns)
# df.head()


In [3]:
# CLASSIFICATION 'Code_type_local'

# TRAINING

# Import train_test_split function
from sklearn.model_selection import train_test_split
# ########### Random forest
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, ConfusionMatrixDisplay
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from scipy.stats import randint

# Random forest mieux mais 8min de calcul


X = trainning_df.drop(columns=['Code_type_local', 'Commune', 'Section', 'No_plan', 'Nature_culture',
            'Departement', 'Code_postal', 'day', 'month', 'year', 'latitude', 'longitude'])

y = trainning_df['Code_type_local']  # Target variable

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=1)


modele_decision_tree = DecisionTreeClassifier(random_state=42)

modele_decision_tree.fit(X_train, y_train)
y_pred = modele_decision_tree.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)


Accuracy: 0.9287105570705103


In [5]:
# Use previous model to fill 'Code_type_local' == NaN

# Réimporter les données de base, tous ceux qui ont Code_type_local == NaN, puis appliquer le modèle le classif 

# Sélectionnez les individus avec Code_type_local manquant
individus_manquants = df_with_nan[df_with_nan['Code_type_local'].isna()]
print(individus_manquants)

X_manquants = individus_manquants.drop(columns=['Code_type_local', 'Commune', 'Section', 'No_plan', 'Nature_culture',
                                                'Departement', 'Code_postal', 'day', 'month', 'year', 'latitude', 'longitude'])
print(X_manquants)


y = individus_manquants['Code_type_local']

# Prédisez Code_type_local pour les individus manquants
predictions_code_type_local = modele_decision_tree.predict(X_manquants)

# Remplacez les valeurs manquantes dans le DataFrame original avec les prédictions
df_with_nan.loc[df_with_nan['Code_type_local'].isna(), 'Code_type_local'] = predictions_code_type_local


df_with_nan.head()


       No_disposition  Valeur_fonciere Code_postal                  Commune  \
1                   2          2500.00        3100                MONTLUCON   
5                   1          2419.00        2310                   PAVANT   
7                   1           519.00        1200         INJOUX GENISSIAT   
8                   1         10000.00        2860  BRUYERES ET MONTBERAULT   
10                  1        242000.00        1260            HAUT VALROMEY   
...               ...              ...         ...                      ...   
47182               1         78000.00        3320               LE VEURDRE   
47183               1        160000.00        2510                   ETREUX   
47186               1         60000.00        3320               LE VEURDRE   
47188               1          2000.00        2270          CRECY SUR SERRE   
47190               1         26490.75        2310                  DOMPTIN   

      Section  No_plan  Nombre_de_lots  Code_type_l

ValueError: Input X contains NaN.
DecisionTreeClassifier does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

In [ ]:
# Concaténer les 2 dataframes (sans les 'Code_type_local' manquants et avec)

df = pd.concat([df_without_nan, df_with_nan])
df.head()

In [ ]:
# Replace last empty variables

median_surface_by_type = df.groupby('Code_type_local')[
    'Surface_reelle_bati'].median()
# Remplacez les NaN dans "Surface_reelle_bat" par la médiane correspondante basée sur "Code_type_local"
df['Surface_reelle_bati'] = df['Surface_reelle_bati'].fillna(
    df['Code_type_local'].map(median_surface_by_type))

median_surface_by_type = df.groupby('Code_type_local')[
    'Nombre_pieces_principales'].median()
# Remplacez les NaN dans "Surface_reelle_bat" par la médiane correspondante basée sur "Code_type_local"
df['Nombre_pieces_principales'] = df['Nombre_pieces_principales'].fillna(
    df['Code_type_local'].map(median_surface_by_type))

# Removing useless variables
df = df.drop(columns=["Nature_culture", "Surface_terrain"])

In [ ]:
pourcentages_nan = (df.isna().sum() / len(df)) * 100
print(pourcentages_nan)
print(len(df))

df.head()

In [ ]:
# Adding Variable 'Prix_moyen_m2'

import numpy as np

# Groupez les données par 'Commune' et calculez la somme de 'Surface_reel' et 'Valeur_fonciere' pour chaque commune
groupe_commune = df.groupby('Commune').agg(
    {'Surface_reelle_bati': 'sum', 'Valeur_fonciere': 'sum'})

# Calculez le prix moyen au mètre carré par commune
groupe_commune['Prix_moyen_m2'] = groupe_commune['Valeur_fonciere'] / \
    groupe_commune['Surface_reelle_bati']


# Fusionnez le DataFrame original avec le DataFrame groupe_commune sur la colonne 'Commune'
df = pd.merge(df, groupe_commune[['Prix_moyen_m2']],
              left_on='Commune', right_index=True, how='left')



# Remplace les inf par NaN
df = df.replace([np.inf, -np.inf], np.nan)
# Supprime les lignes avec NaN dans 'Prix_moyen_m2'
df = df.dropna(subset=['Prix_moyen_m2'], how='any')

df.info()


In [ ]:
# Export final data

df.to_pickle('../data/final_data.pkl')
